# IMDB Project 3
- Juliana Sahagun
- 08/17/22

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import json,os,time
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook
# Imports for MySQL Requirements
import pymysql
from sqlalchemy import create_engine
from sqlalchemy_utils import create_database, database_exists


In [2]:
# Download files
basics_url= "https://datasets.imdbws.com/title.basics.tsv.gz"
ratings_url= "https://datasets.imdbws.com/title.basics.tsv.gz"
akas_url= "https://datasets.imdbws.com/title.akas.tsv.gz"

In [3]:
# Loading the data
df_basics = pd.read_csv(basics_url,sep='\t', low_memory=False)
df_akas= pd.read_csv(akas_url, sep='\t', low_memory=False)
df_ratings= pd.read_csv(ratings_url,sep='\t', low_memory=False)

# Part 1

In [4]:
df_basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [5]:
df_ratings.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [6]:
df_akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


## Cleaning/Filtering

Basics Preprocessing

In [7]:
# Replace "\N" with np.nan
df_basics = df_basics.replace({'\\N':np.nan})

In [8]:
# Remove movies having missing values for runtimeMinutes and genre
df_basics= df_basics.dropna(subset=['runtimeMinutes','genres','startYear'])

startYear also has null values that should be removed because it interferes with keep all movies with the start year 2000-2022

In [9]:
# Include only full-length movies (titleType = "movie")
df_basics = df_basics.loc[df_basics['titleType']=='movie']

In [10]:
# Eliminate movies that include "Documentary" in genre
doc= df_basics['genres'].str.contains('documentary', case=False)
df_basics = df_basics[~doc]

In [16]:
# Keep startYear 2000-2022
df_basics= df_basics.loc[(df_basics['startYear'] >=2000) & (df_basics['startYear'] <=2021)]

TypeError: '>=' not supported between instances of 'str' and 'int'

In [ ]:
df_basics.info()

Akas Preprocessing

In [12]:
#Keep only movies that were released in the United States
df_akas = df_akas.loc[df_akas['region']=='US']

In [13]:
# Replace "\N" with np.nan
df_akas=df_akas.replace({'\\N': np.nan})

In [14]:
df_akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1346925 entries, 5 to 33078856
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1346925 non-null  object
 1   ordering         1346925 non-null  int64 
 2   title            1346925 non-null  object
 3   region           1346925 non-null  object
 4   language         3701 non-null     object
 5   types            963801 non-null   object
 6   attributes       44851 non-null    object
 7   isOriginalTitle  1345550 non-null  object
dtypes: int64(1), object(7)
memory usage: 92.5+ MB


Ratings Preprocessing

In [ ]:
# Replace "\N" with np.nan
df_ratings = df_ratings.replace({'\\N':np.nan})

In [ ]:
#Filtering one dataframe based on another
keepers = df_basics['tconst'].isin(df_akas['titleId'])
df_basics =df_basics[keepers]
df_basics

In [ ]:
# Make folders for data
import os
os.makedirs('Data/',exist_ok=True) 

# Confirm folder created
os.listdir("Data/")


In [ ]:
# Save current dataframe to file
df_basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)
df_akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)
df_ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

In [ ]:
# Open saved file and preview again
df_basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
df_basics.head()



In [ ]:
df_ratings=pd.read_csv("Data/title_ratings.csv.gz", low_memory =False)
df_ratings.head()

In [ ]:
df_akas=pd.read_csv("Data/title_akas.csv.gz",low_memory=False)
df_akas.head()

# Part 2

In [ ]:
# Open and read file
with open('/Users/julia/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
login.keys()

In [ ]:
tmdb.API_KEY =  login['api-key']

Function below adds the certification (MPGG Rating) to movie.info

In [ ]:
# Code adapted from the LP
def get_movie_and_rating(movie_id):    
    
    movie = tmdb.Movies(movie_id)
    movie_info = movie.info()
    releases = movie.releases()

    for c in releases['countries']:
        if c['iso_3166_1' ] =='US':
            movie_info['certification'] = c['certification']
    return movie_info

In [ ]:
# Test function on these two movies to make sure it runs correctly
test = get_movie_and_rating("tt0848228")
test

In [ ]:
test2 = get_movie_and_rating("tt0332280")
test2

In [ ]:
# Create a folder in data for each year
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

In [ ]:
YEARS_TO_GET =[2000,2001]

In [ ]:
def write_json(new_data, filename): 
    """Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""    
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [ ]:
def get_movie_with_rating(movie_id):
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    # save the .info .releases dictionaries
    movie_info = movie.info()
    releases = movie.releases()
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            ## save a "certification" key in the info dict with the certification
            movie_info['certification'] = c['certification']
    return movie_info

In [ ]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET,desc='YEARS',position=0):
    
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    
    # If it does not exist: create it
    if file_exists == False:
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
    
    #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()#.to_list()
    
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
            
            #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                      desc=f'Movies from {YEAR}',
                                      position=1,
                                      leave=True):
            # Attempt to retrieve then data for the movie id
            try:
                temp = get_movie_with_rating(movie_id)  #This uses your pre-made function
                # Append/extend results to existing file using a pre-made function
                write_json(temp,JSON_FILE)
                # Short 20 ms sleep to prevent overwhelming server
                time.sleep(0.02)

            # If it fails,  make a dict with just the id and None for certification.
            except Exception as e:
                continue

    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

# Part 3

## **Normalizing Genres**

In [ ]:
# Load data again
# Load title basics file 
basics_df = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)

# Load title rating file 
rating_df = pd.read_csv("Data/title_ratings.csv.gz", low_memory = False)

# Load in your csv.gz's of results for combined years
combined = pd.read_csv("Data/tmdb_results_combined.csv.gz", low_memory = False)

In [ ]:
# Convert the single string of genres from title basics into 2 new tables

# create a col with a list of genres
basics_df['genres_split'] = basics_df['genres'].str.split(',')
basics_df

In [ ]:
# exploding the column of lists
exploded = basics_df.explode('genres_split')
exploded

In [ ]:
# saving the unique values from the exploded column
unique_genres = sorted(exploded['genres_split'].dropna().unique())
unique_genres

In [ ]:
# Create a new title_genres table
title_genres = exploded[['tconst','genres_split']].copy()
title_genres.head()

In [ ]:
#  Create a genre mapper dictionary to replace string genres with integers
genre_ints = range(len(unique_genres))
genre_map = dict(zip(unique_genres, genre_ints))
genre_map

In [ ]:
# Create a new genre_id column using the genre_map dictionary with either .map or .replace.
title_genres['genre_id'] = title_genres['genres_split'].map(genre_map)

title_genres = title_genres.drop(columns='genres_split')
title_genres

In [ ]:
# Convert the genre map dictionary into a dataframe
genre = pd.DataFrame({'Genre_Name' : genre_map.keys(),
                     'Genre_Id': genre_map.values()})
genre.head()

In [ ]:
 # Extract certain columns
tmdb = combined[['imdb_id','budget','revenue', 'certification']]
tmdb

## **Discard unnecessary information**

In [ ]:
# Drop following columns
basics_df = basics_df.drop(['originalTitle','isAdult', 'titleType','genres','genres_split'], axis = 1)
basics_df

## **MySQL Database**

In [ ]:
# loading mysql credentials
with open('/Users/julia/.secret/mysql.json') as f:
    login =json.load(f)
login.keys

In [ ]:
# Make connection string
connection_str= f"mysql+pymysql://{login['Login Name']}:{login['password']}@localhost/movies"
engine= create_engine(connection_str)

In [ ]:
# create database
create_database(connection_str)

In [ ]:
# Transform to tables
basics_df.to_sql('title_basics', engine, if_exists = 'replace')
rating_df.to_sql('title_rating', engine, if_exists = 'replace')
title_genres.to_sql('title_genres', engine, if_exists = 'replace')
genre.to_sql('genres', engine, if_exists = 'replace')
tmdb.to_sql('tmdb_data', engine, if_exists = 'replace')

**Set primary key for title_basics tables**

In [ ]:
## Example
from sqlalchemy.types import *
## Calculate max string lengths for object columns
key_len = basics_df['tconst'].fillna('').map(len).max()
title_len = basics_df['primaryTitle'].fillna('').map(len).max()
## Create a schema dictonary using Sqlalchemy datatype objects
df_schema = {
    "tconst": String(key_len+1), 
    "primaryTitle": Text(title_len+1),
    'startYear':Float(),
    'endYear':Float(),
    'runtimeMinutes':Integer()}


In [ ]:
# Save to sql with dtype and index=False
basics_df.to_sql('title_basics',engine,dtype=df_schema,if_exists='replace',index=False)

In [ ]:
engine.execute('ALTER TABLE title_basics ADD PRIMARY KEY (`tconst`);')

In [ ]:
# show the first 5 rows of the table using a SQL query
q = """
SELECT *
FROM title_basics
LIMIT 5;"""
pd.read_sql(q, engine)

**Set primary key for ratings tables**

In [ ]:
# Calculate max string lengths for object columns
key_len = rating['tconst'].fillna('').map(len).max()

## Create a schema dictonary using Sqlalchemy datatype objects
df_2 = {
    "tconst": String(key_len+1),
    'averageRating':Float(),
    'numVotes':Integer()}

In [ ]:
# Save to sql with dtype and index=False
rating.to_sql('title_rating',engine,dtype=df_2,if_exists='replace',index=False)

In [ ]:
# update the table and set the primary key
engine.execute('ALTER TABLE title_rating ADD PRIMARY KEY (`tconst`);')

In [ ]:
# show the first 5 rows of the table using a SQL query
q = """
SELECT *
FROM title_rating
LIMIT 5;"""
pd.read_sql(q, engine)

**Set primary key for genre tables**

In [ ]:
# Set the dataframe index and use index=True 
genre.set_index('Genre_Id').to_sql('genres',engine,index=True, if_exists = 'replace')

In [ ]:
# show the first 5 rows of the table using a SQL query
q = """
SELECT *
FROM genres
LIMIT 5;"""
pd.read_sql(q, engine)

**Set primary key for tmdb tables**

In [ ]:
# Calculate max string lengths for object columns
key_len = tmdb['imdb_id'].fillna('').map(len).max()
cert_len = tmdb['certification'].fillna('').map(len).max()

## Create a schema dictonary using Sqlalchemy datatype objects
df_3 = {
    "imdb_id": String(key_len+1),
    'budget':Float(),
    'revenue':Float(),
    "certification":Text(cert_len+1)}

In [ ]:
# Save to sql with dtype and index=False
tmdb.to_sql('tmdb_data',engine,dtype=df_3,if_exists='replace',index=False)

In [ ]:
# update the table and set the primary key
engine.execute('ALTER TABLE tmdb_data ADD PRIMARY KEY (`imdb_id`);')

In [ ]:
# show the first 5 rows of the table using a SQL query
q = """
SELECT *
FROM tmdb_data
LIMIT 5;"""
pd.read_sql(q, engine)

In [ ]:
# Show all tables
q = """SHOW TABLES;"""
pd.read_sql(q, engine)